## 07: Interactive Facility Scenario Explorer

**Goal:** To create a simple, user-driven tool for exploring "what-if" scenarios in facility planning. Static maps are useful, but interactive tools allow stakeholders to get immediate feedback on their ideas.

This notebook demonstrates how to:
1.  Use `ipywidgets` and `ipyleaflet` to create an interactive map directly within Jupyter.
2.  Display existing facilities and a set of demand points.
3.  Allow a user to **click on the map to propose a new facility location**.
4.  Dynamically recalculate which demand points would be served by this new facility and update the map instantly.

### 1. Setup and Library Imports

You might need to install these libraries first:
`pip install ipywidgets ipyleaflet`
Then, enable the extension:
`jupyter nbextension enable --py widgetsnbextension`

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from ipyleaflet import Map, Marker, GeoData, basemaps, LayersControl
import ipywidgets as widgets
from IPython.display import display

# Helper function for styling GeoData layers
def style_layer(feature, **kwargs):
    return {'color': 'black', 'weight': 1, 'fillColor': 'blue', 'fillOpacity': 0.5}

### 2. Create Synthetic Data

We'll generate a set of existing facilities and a grid of demand points representing residential areas.

In [ ]:
np.random.seed(123)

# Existing facilities
existing_facilities = gpd.GeoDataFrame(
    {'name': ['Existing Site A', 'Existing Site B']},
    geometry=[Point( -1.85, 51.56), Point(-1.75, 51.58)],
    crs="EPSG:4326"
)

# A grid of demand points
xx, yy = np.meshgrid(np.linspace(-1.9, -1.7, 10), np.linspace(51.55, 51.6, 10))
demand_points = gpd.GeoDataFrame(
    geometry=[Point(x, y) for x, y in zip(xx.ravel(), yy.ravel())],
    crs="EPSG:4326"
)
demand_points['id'] = range(len(demand_points))

# Keep track of all facilities (existing and proposed)
all_facilities = existing_facilities.copy()

### 3. Core Logic: Allocate Demand and Create Map Layers

In [ ]:
def allocate_demand_and_get_layers(facilities_gdf, demand_gdf):
    """Calculates which facility serves each demand point and prepares layers for the map."""
    if facilities_gdf.empty:
        # If there are no facilities, color all demand points as unserved
        demand_gdf['color'] = 'gray'
        demand_layer = GeoData(geo_dataframe=demand_gdf, style_callback=lambda feat: {'color': feat['properties']['color']}, name='Demand')
        return [demand_layer], []

    # Project to a meter-based CRS for distance calculation
    facilities_proj = facilities_gdf.to_crs(epsg=32630)
    demand_proj = demand_gdf.to_crs(epsg=32630)

    # For each demand point, find the nearest facility
    allocations = []
    for idx, demand_point in demand_proj.iterrows():
        distances = facilities_proj.distance(demand_point.geometry)
        closest_idx = distances.idxmin()
        allocations.append(closest_idx)
    
    demand_proj['allocated_to'] = allocations

    # Create colors for visualization
    unique_facilities = demand_proj['allocated_to'].unique()
    colors = plt.cm.get_cmap('tab10', len(unique_facilities))
    color_map = {fac_idx: plt.matplotlib.colors.to_hex(colors(i)) for i, fac_idx in enumerate(unique_facilities)}
    demand_proj['color'] = demand_proj['allocated_to'].map(color_map)

    # Convert back to WGS84 for ipyleaflet
    demand_final = demand_proj.to_crs(epsg=4326)

    # Create map layers
    demand_layer = GeoData(geo_dataframe=demand_final, 
                           point_style={'radius': 5, 'color': 'black', 'weight': 0.5, 'fillColor': 'red', 'fillOpacity': 0.7},
                           style_callback=lambda feat: {'fillColor': feat['properties']['color']},
                           name='Demand')
    
    facility_markers = [Marker(location=(g.y, g.x), draggable=False, title=name) 
                        for g, name in zip(facilities_gdf.geometry, facilities_gdf.name)]
    
    return [demand_layer], facility_markers


### 4. Build the Interactive Map

This is where we tie everything together. We'll create a map, a button, and an output widget to display messages. The core logic is in the `handle_interaction` function, which is called every time the user clicks on the map.

In [ ]:
# Initialize map centered on our area of interest
m = Map(center=(51.575, -1.82), zoom=12, basemap=basemaps.CartoDB.Positron)

# Create an output widget for logs
out = widgets.Output(layout={'border': '1px solid black'})

# Store map layers globally so we can remove them
current_demand_layer = None
current_facility_markers = []

def update_map():
    global all_facilities, demand_points, m, current_demand_layer, current_facility_markers
    
    # Remove old layers
    if current_demand_layer in m.layers:
        m.remove_layer(current_demand_layer)
    for marker in current_facility_markers:
        if marker in m.layers:
            m.remove_layer(marker)
            
    # Get new layers
    [demand_layer], facility_markers = allocate_demand_and_get_layers(all_facilities, demand_points)
    
    # Add new layers
    m.add_layer(demand_layer)
    for marker in facility_markers:
        m.add_layer(marker)
        
    # Update global references
    current_demand_layer = demand_layer
    current_facility_markers = facility_markers

def handle_interaction(**kwargs):
    global all_facilities
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        with out:
            out.clear_output()
            new_name = f"Proposed Site {len(all_facilities) - 1}"
            print(f"Adding new facility '{new_name}' at {coords}")
            new_facility = gpd.GeoDataFrame(
                {'name': [new_name]},
                geometry=[Point(coords[1], coords[0])], # Note: Leaflet gives (lat, lon)
                crs="EPSG:4326"
            )
            all_facilities = pd.concat([all_facilities, new_facility], ignore_index=True)
            update_map()

# Button to reset the scenario
reset_button = widgets.Button(description="Reset Scenario")
def on_reset_button_clicked(b):
    global all_facilities
    with out:
        out.clear_output()
        print("Resetting to initial state.")
        all_facilities = existing_facilities.copy()
        update_map()
reset_button.on_click(on_reset_button_clicked)

# Attach the click handler to the map
m.on_interaction(handle_interaction)

# Initial map state
update_map()

# Display the final layout
display(widgets.VBox([widgets.HTML("<h3>Click on the map to add a new facility</h3>"), m, reset_button, out]))

### 5. How to Use

1.  Run all the cells above.
2.  The map will display showing the initial state: two existing facilities and the demand points colored according to which of the two is closer.
3.  **Click anywhere on the map.** A new marker will appear, representing a proposed facility.
4.  The colors of the demand points will instantly update, showing the new catchment areas. You can see how many demand points are now served by your proposed site.
5.  Click the **Reset Scenario** button to remove all proposed sites and return to the initial state.